In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Convolution3D(nn.Module):
    def __init__(self):
        super(Convolution3D,self).__init__()
        
        # 3*3*3 Conv3d 
        self.conv1 = nn.Conv3d(in_channels = 3, out_channels = 64, kernel_size=(3,3,3),padding=(1,0,0))
        self.bn1 = nn.BatchNorm3d(num_features = 64)
        # 3D MaxPooling
        self.mp1 = nn.MaxPool3d(kernel_size = (1,2,2), stride=(1,2,2), padding=(0,0,0))
        # 3D MaxPooling
        self.mp2 = nn.MaxPool3d(kernel_size =(1,2,2), stride=(1,2,2), padding=(0,0,0))
        # 3*3*3 Conv3d
        self.conv2 = nn.Conv3d(in_channels = 64, out_channels = 64,kernel_size=3, padding = (1,0,0))
        self.bn2 = nn.BatchNorm3d(num_features = 64)
        # Residual Convolution
        self.resconv1 = nn.Conv3d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = (1,1,1))
        self.ap1 = nn.AvgPool3d(kernel_size = 1, stride =1, padding = (0,0,0))
        self.resbn1 = nn.BatchNorm3d(num_features = 64)
        # 3*3*3 Conv3d
        self.conv3 = nn.Conv3d(in_channels = 64, out_channels = 64,kernel_size=3, padding = (1,0,0))
        self.bn3 = nn.BatchNorm3d(num_features = 64)
        # Residual Convolution
        self.resconv2 = nn.Conv3d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = (1,1,1))
        self.ap2 = nn.AvgPool3d(kernel_size = 1, stride=1, padding = (0,0,0) )
        self.resbn2 = nn.BatchNorm3d(num_features = 64)
        # 3*3*3 Conv3d
        self.conv4 = nn.Conv3d(in_channels = 64, out_channels = 8, kernel_size=(3,3,3), padding=(1,0,0))
        self.bn4 = nn.BatchNorm3d(num_features = 8)
        # 3*3*3 Conv3d
        self.conv5 = nn.Conv3d(in_channels = 8, out_channels = 8, kernel_size=(3,3,3), padding=(1,0,0))
        self.bn5 = nn.BatchNorm3d(num_features = 8)
        # 3*3*3 Conv3d
        self.conv6 = nn.Conv3d(in_channels = 8, out_channels = 8, kernel_size=(3,3,3), padding=(1,0,0))
        self.bn6 = nn.BatchNorm3d(num_features = 8)
        
        self.Flatten1 = nn.Flatten(start_dim=2)
        
        self.LSTM1 = nn.LSTM(input_size = 10488, hidden_size = 64, num_layers = 1, batch_first=True)
        self.LSTM2 = nn.LSTM(input_size = 64, hidden_size = 16, num_layers = 1, batch_first=True)
        
        self.fc1 = nn.Linear(in_features = 16, out_features = 512, bias=True)
        self.fc2 = nn.Linear(in_features = 512, out_features = 128, bias=True)
        self.fc3 = nn.Linear(in_features = 128, out_features = 64, bias=True)
        self.fc4 = nn.Linear(in_features = 64, out_features = 16, bias=True)
        self.fc5 = nn.Linear(in_features = 16, out_features = 1, bias=True)


    def forward(self, x):
        # 3*3*3 3D Conv 3
        x = F.relu(self.bn1(self.conv1(x)))
        # 3D Max Pooling
        x = self.mp1(x)
        # 3D Max Pooling
        x = self.mp2(x)
        # 3*3*3 3D Conv 64
        x = F.relu(self.bn2(self.conv2(x)))
        # ResNet - 3*3*3 3D Conv 64
        Residual = x
        Res_Output = F.relu(self.resbn1(self.resconv1(x)))
        x = F.relu(Residual + Res_Output)
        x = self.ap1(x)
        # 3*3*3 3D Conv 64
        x = F.relu(self.bn3(self.conv3(x)))
        # ResNet - 3*3*3 3D Conv 64
        Residual = x
        Res_Output = F.relu(self.resbn2(self.resconv2(x)))
        x = F.relu(Residual + Res_Output)
        del Residual
        del Res_Output
        x = self.ap2(x)
        # 3*3*3 3D Conv 8
        x = F.relu(self.bn4(self.conv4(x)))
        # 3*3*3 3D Conv 8
        x = F.relu(self.bn5(self.conv5(x)))
        # 3*3*3 3D Conv 8
        x = F.relu(self.bn6(self.conv6(x)))
        
        # LSTM 64
        x = x.permute([0,2,1,3,4])
        x = self.Flatten1(x)
        x = torch.tanh(self.LSTM1(x)[0])        
        # LSTM 16
        x = torch.tanh(self.LSTM2(x)[0])

        # FC 512
        x = x.permute(1,0,2)
        x = F.relu(self.fc1(x))
        # FC 128
        x = F.relu(self.fc2(x))
        # FC 64
        x = F.relu(self.fc3(x))
        # FC 16
        x = F.relu(self.fc4(x))
        # FC 1
        angle = self.fc5(x)

        return angle

DA FARE: 

se il batch ha la shape: (batch, C, H, W) allora settare 
`batch_size = 25` e per ogni batch fare il reshape: 


```
for idx, batch in enumerate(self.data):
    batch = batch.reshape((5, 5, 3, 240, 400))
    ...
    ...
    ...
```

assumendo che la foto sia 3x240x400 post preprocess. 

Per esempio su un dato finto >> runna sotto


# Altro

In [ ]:
agent = Convolution3D()

Test pre scoprire che non possiamo decidere di stackare i frame

In [ ]:
x = torch.rand((5,5,120,320,3))
print(x.shape)
x = x.permute((0,4,1,2,3))
print(x.shape)

In [ ]:
y = agent.forward(x)

In [ ]:
y.shape

Test supponendo che abbiamo un batch da 25 e se reshape funge per fare 5 e 5

In [ ]:
x = torch.rand((25, 3, 240, 400))
y = x.reshape((5, 5, 3, 240, 400))

In [ ]:
y[0].shape, x[:5].shape

In [ ]:
torch.all(x[:5] == y[0])